In [1]:
import numpy as np
from scipy.linalg import logm
from chapter4 import PrismaticJoint, RevoluteJoint, Robot, extract_screw_from_logm

In [2]:
joint1 = RevoluteJoint(name='joint1', position=np.array([1, 0, 0]), rotation_axis=np.array([0, 0, 1]))
joint2 = RevoluteJoint(name='joint2', position=np.array([2, 0, 0]), rotation_axis=np.array([0, 0, 1]))
joint3 = PrismaticJoint(name='joint3', position=np.array([3, 0, 0]),translation_axis=np.array([0, 0, 1]))
joint4 = RevoluteJoint(name='joint4', position=np.array([3, 0, 0]), rotation_axis=np.array([0, 0, 1]))

base_position = np.array([0, 0, 0])
base_orientation = np.eye(3)
end_effector_position = np.array([3, 0, 0])
end_effector_orientation = np.eye(3)

In [3]:
rb = Robot()

rb.register_joint(joint=joint1)
rb.register_joint(joint=joint2)
rb.register_joint(joint=joint3)
rb.register_joint(joint=joint4)

rb.base_frame.update_orientation(base_position)
rb.base_frame.update_orientation(base_orientation)
rb.end_effector_frame.update_position(end_effector_position)
rb.end_effector_frame.update_orientation(end_effector_orientation)

In [4]:
s_list = []
b_list = []
for key in rb.joints:
    screwS = rb.compute_screw_axis(key)
    screwB = rb.compute_screw_axis(key, False)
    print(key)
    print(f'S: {np.round(screwS,2)}')
    print(f'B: {np.round(screwB,2)}')
    print()

joint1
S: [ 0.  0.  1. -0. -1. -0.]
B: [ 0.  0.  1. -0.  2. -0.]

joint2
S: [ 0.  0.  1. -0. -2. -0.]
B: [ 0.  0.  1. -0.  1. -0.]

joint3
S: [0. 0. 0. 0. 0. 1.]
B: [0. 0. 0. 0. 0. 1.]

joint4
S: [ 0.  0.  1. -0. -3. -0.]
B: [ 0.  0.  1. -0. -0. -0.]



In [5]:
thetas = {'joint1':-np.pi/2,
          'joint2': np.pi/2,
          'joint3': 2,
          'joint4':-np.pi/4}

T = rb.compute_total_transform(thetas)
print(np.round(T, 2))

[[ 0.71  0.71  0.    2.  ]
 [-0.71  0.71  0.   -1.  ]
 [ 0.    0.    1.    2.  ]
 [ 0.    0.    0.    1.  ]]


In [6]:
names = ['joint1','joint2','joint3']
joint1 = RevoluteJoint(name='joint1', position=np.array([0, 0, 0]), rotation_axis=np.array([0, 0, 1]))
joint2 = RevoluteJoint(name='joint2', position=np.array([1, 0, 0]), rotation_axis=np.array([0, 0, 1]))
joint3 = RevoluteJoint(name='joint3', position=np.array([2, 0, 0]), rotation_axis=np.array([0, 0, 1]))

base_position = np.array([0, 0, 0])
base_orientation = np.eye(3)
end_effector_position = np.array([3, 0, 0])
end_effector_orientation = np.eye(3)

rb = Robot()

rb.register_joint(joint=joint1)
rb.register_joint(joint=joint2)
rb.register_joint(joint=joint3)

rb.base_frame.update_orientation(base_position)
rb.base_frame.update_orientation(base_orientation)
rb.end_effector_frame.update_position(end_effector_position)
rb.end_effector_frame.update_orientation(end_effector_orientation)


T_sd = np.array([[-0.585, -0.811, 0, 0.076],
                 [0.811, -0.585, 0, 2.608],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

In [9]:
thetas = np.array([np.pi/4, np.pi/4, np.pi/4])
theta_dict = dict(zip(names, thetas))
epsilon = 1e-6  # Tolerance for convergence check

for i in range(100):
    theta_dict = dict(zip(names, thetas))
    T_sb = rb.compute_total_transform(theta_dict, isBase=True)
    T_new = np.matmul(np.linalg.inv(T_sb), T_sd)

    T_new_log = logm(T_new)
    w, v = extract_screw_from_logm(T_new_log)
    screw_axis = np.hstack([w, v])

    J = rb.compute_jacobian_base(theta_dict, isBase=False)
    delta_theta = np.matmul(np.linalg.pinv(J), screw_axis)

    # Check for convergence
    if np.linalg.norm(delta_theta) < epsilon:
        break

    thetas = thetas + delta_theta


In [10]:
thetas

array([0.92519858, 0.58622306, 0.68427422])